In [1]:
using Pkg

cd(@__DIR__)
Pkg.activate("../..")
using Metal

  Activating project at `~/Documents/Github/flexibleDSM`


In [15]:
using KernelAbstractions
using Adapt
using Metal

In [37]:
@kernel function mul2_kernel(A)
    i = @index(Global)
    A[i]=2 * A[i]
end

mul2_kernel (generic function with 4 methods)

In [82]:
dev = CPU()
A=ones(1024*8,1024*8)
@time ev = mul2_kernel(dev,64)(A,ndrange=size(A))

  0.035161 seconds (26 allocations: 912 bytes)


In [83]:
KernelAbstractions.synchronize(dev)

In [84]:
all(A.==2.0)

true

In [85]:
A.=1.0


8192×8192 Matrix{Float64}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 ⋮                        ⋮              ⋱  ⋮                        ⋮    
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0 

In [86]:
backend=get_backend(A)

CPU(false)

In [87]:
ev = mul2_kernel(backend,64)(A,ndrange=size(A))

In [88]:
all(A.==2.0)

true

In [93]:
A=ones(1024*8,1024*8)

A_gpu= Adapt.adapt(MetalBackend(), Float32.(A))

8192×8192 MtlMatrix{Float32, Metal.PrivateStorage}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 ⋮                        ⋮              ⋱  ⋮                        ⋮    
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0 

In [94]:
dev =MetalBackend()

#ev = mul2_kernel(dev,64)(A_gpu,ndrange=size(A_gpu))

@time ev=mul2_kernel(dev,64)(A_gpu; ndrange=size(A_gpu))
#KernelAbstractions.wait(ev)
@print ev
KernelAbstractions.synchronize(dev)

  0.000231 seconds (70 allocations: 2.250 KiB)
nothing

In [95]:
A_gpu

8192×8192 MtlMatrix{Float32, Metal.PrivateStorage}:
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 ⋮                        ⋮              ⋱  ⋮                        ⋮    
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0 

In [96]:
新A= Adapt.adapt(CPU(), A_gpu)

8192×8192 Matrix{Float32}:
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0  …  2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 ⋮                        ⋮              ⋱  ⋮                        ⋮    
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0     2.0  2.0  2.0  2.0  2.0  2.0  2.0
 2.0  2.0  2.0  2.0  2.0  2.0  2.0  2.0 

In [102]:

@kernel function piecewiseProduct!(a,b,result)
    i = @index(Global)
    result[i]=a[i]*b[i]
end

function GPUsum(gpuArray)
    return KernelAbstractions.sum(gpuArray)
end


GPUsum (generic function with 1 method)

In [111]:
result=similar(A_gpu)
piecewiseProduct!(MetalBackend(),64)(A_gpu,A_gpu,result; ndrange=size(A_gpu))
@show innerPro=GPUsum(result)

2.6843546f8

In [114]:
4*8192*8192 === Int(innerPro)

true

In [115]:
@kernel function piecewiseProduct!(a,b,result)
    i = @index(Global)
    result[i]=a[i]*b[i]
end

@kernel function piecewiseProduct!(a,b,c,result)
    i = @index(Global)
    result[i]=a[i]*b[i]*c[i]
end

@kernel function piecewiseProduct!(a,b,c,d,result)
    i = @index(Global)
    result[i]=a[i]*b[i]*c[i]*d[i]
end


In [117]:
piecewiseProduct!(MetalBackend(),64)(A_gpu,A_gpu,A_gpu,result; ndrange=size(A_gpu))
 innerPro=GPUsum(result)

5.368709f8

In [118]:
Int(innerPro)

536870912